In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [2]:
data_dir = "/media/mountHDD2/khoibaocon"
print(os.listdir(data_dir))

['TrainingSet3', 'Label.csv', 'alldata', 'TrainingSet1', 'single_label.csv', 'TrainingSet2']


In [3]:
main_df = pd.read_csv(data_dir + "/Label.csv")
main_df.shape

(6877, 4)

In [4]:
main_df["First_label"].value_counts()

First_label
5    1695
2    1098
1     918
8     826
3     704
7     653
6     574
4     207
9     202
Name: count, dtype: int64

In [5]:
main_df["Second_label"].value_counts()

Second_label
5.0    162
2.0    123
7.0     47
6.0     42
8.0     41
4.0     28
3.0     18
9.0     16
Name: count, dtype: int64

In [6]:
main_df["Third_label"].value_counts()

Third_label
9.0    2
8.0    2
6.0    1
4.0    1
Name: count, dtype: int64

In [7]:
d = (os.listdir("/media/mountHDD2/khoibaocon/alldata"))
print(len(d))

6877


In [8]:
first_lb_df = main_df[main_df["First_label"].notnull()]
first_fns = first_lb_df["Recording"].values.tolist()

In [9]:
second_lb_df = main_df[main_df["Second_label"].notnull()]
second_fns = second_lb_df["Recording"].values.tolist()

third_lb_df = main_df[main_df["Third_label"].notnull()]
third_fns = third_lb_df["Recording"].values.tolist()

In [10]:
first_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in first_fns]
second_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in second_fns]
third_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in third_fns]
all_mat_paths = first_mat_paths + second_mat_paths + third_mat_paths
print(len(all_mat_paths))

7360


In [11]:
class Redu(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size):
        super().__init__()
        
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        
        self.conv1 = nn.Sequential(
            nn.MaxPool1d(4),
            nn.Conv1d(self.in_channel, self.out_channel, 1, 1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(self.in_channel, self.out_channel, self.kernel_size, 1),
            nn.LeakyReLU(),
            nn.BatchNorm1d(self.out_channel),
            nn.Dropout(0.2),
            nn.Conv1d(self.out_channel, self.out_channel, self.kernel_size, 4, )
        )
        self.conv3 = nn.Sequential(
            nn.LeakyReLU(),
            nn.BatchNorm1d(self.out_channel),
            nn.Dropout(0.2),
        )
    
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x_res = x1 + x2
        x3 = self.conv3(x_res)
        
        return x3, x_res

In [12]:
conv_block1 = nn.Sequential(
			nn.Conv1d(12, 64, 16, stride=1),
			nn.LeakyReLU(0.3),
			nn.BatchNorm1d(64),
		)

In [21]:
test = torch.randn(1, 12, 1400)
test = conv_block1(test)
model = Redu(64, 128, 3)
a = model(test)
print(len(a[1][0]))

128


In [22]:
class LSAT(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.in_channel = in_channel
        self.out_channel = out_channel
        
        self.conv1 = nn.Sequential(
            nn.LSTM(320, ),
            nn.BatchNorm1d(self.out_channel),
            nn.Dropout(0.5),
            nn.LSTM(),
            nn.BatchNorm1d(self.out_channel),
            nn.Dropout(0.5),                 
        )
        
        self.conv2 = nn.Sequential(
            nn.Attention(),
            nn.BatchNorm1d(self.out_channel),
            nn.Dropout(0.5),
        )
        
        self.conv3 = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(0.3),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.3),
            nn.Linear(32, 9),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        
        return x3    
    